In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
projectdir = "/home/jovyan/work/MED_Fall"
workdir = "/home/jovyan/work"

In [3]:
import os
os.environ["PYTHONPATH"]

'/home/jovyan/work/MED_Fall'

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

from vision.pre_processing.load_frames_from_videos import FramesExtractor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from utils import listdir_nohidden_sorted as lsdir

2022-07-25 11:17:39.044103: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
tf.config.list_physical_devices("GPU")

2022-07-25 11:17:40.985322: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-25 11:17:40.985361: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 6d2e41cc007a
2022-07-25 11:17:40.985370: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 6d2e41cc007a


[]

2022-07-25 11:17:40.985521: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-07-25 11:17:40.985538: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-07-25 11:17:40.985545: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.129.6 does not match DSO version 510.47.3 -- cannot find working devices in this configuration


## Testing VGG16 Fine Tuning

In [ ]:
IMG_SIZE = (224, 224, 3)

In [ ]:
feature_extractor = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=IMG_SIZE,
    pooling='avg',
)

In [ ]:
feature_extractor.summary()

In [ ]:
import numpy as np

In [ ]:
sample = np.ones(shape=IMG_SIZE)
sample = np.expand_dims(sample, axis=0)

In [ ]:
sample.shape

In [ ]:
# sample=preprocess_input(sample)
# sample.shape
# sample

In [ ]:
out = feature_extractor.predict(sample)

In [ ]:
out.shape

In [ ]:
feature_extractor

In [ ]:
# Freeze four convolution blocks
for layer in feature_extractor.layers[:15]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(feature_extractor.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
x = feature_extractor.output
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=256, activation='relu')(x)
x = Dense(units=3, activation='softmax')(x)

transfer_model = Model(inputs=feature_extractor.input, outputs=x)

In [ ]:
transfer_model.predict(np.expand_dims(X[0], axis=0))

In [ ]:
X = np.ones(shape=(1000, 224, 224, 3))
X.shape

In [ ]:
y = [1] * 300 + [2] * 300 + [3] * 400
y = np.array(y)
y.shape

In [ ]:
transfer_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='acc')

In [ ]:
transfer_model.fit(epochs=10, x=X, y=y)

In [ ]:
last = len(transfer_model.layers)
last

In [ ]:
type(transfer_model.layers)

In [ ]:
#transfer_model.layers[-5:]
from tensorflow.keras.models import Sequential

feature_extractor_tuned = Sequential(transfer_model.layers[:-5])

In [ ]:
feature_extractor_tuned.summary()

In [ ]:
pred = feature_extractor_tuned.predict(np.expand_dims(X[0], axis=0))
pred.shape

# Actual fine tuning pipeline

In [ ]:
extracted_frames_path = (
    "/home/jovyan/work/MED_Fall/vision/vision_dataset/extracted_frames"
)

In [ ]:
start = len(extracted_frames_path) + 1
frames_names = lsdir(extracted_frames_path)
frames_names = list(map(lambda x: x[start:], frames_names))
print(frames_names[:10])

In [ ]:
print(len(frames_names))

In [ ]:
ground_truth_path = "/home/jovyan/work/MED_Fall/vision/vision_dataset/ground_truth"
lsdir(ground_truth_path)

In [ ]:
pd.read_csv(
    "/home/jovyan/work/MED_Fall/vision/vision_dataset/ground_truth/Actor_1_Bed.csv"
)

In [ ]:
dfs = []
for file in lsdir(ground_truth_path):
    df = pd.read_csv(file)
    dfs.append(df)
dataset = pd.concat(dfs, ignore_index=True)

dataset = dataset.iloc[:, 1:]
dataset

In [ ]:
dataset["frame_name"] = pd.Series(frames_names)
dataset

In [ ]:
# select only actor 4
actor_4_dataset = dataset[dataset["frame_name"].str.contains("actor_4")]

In [ ]:
actor_4_dataset.reset_index(drop=True, inplace=True)
actor_4_dataset

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_datagen = train_datagen.flow_from_dataframe(
    actor_4_dataset,
    directory=extracted_frames_path,
    x_col="frame_name",
    y_col="macro_labels",
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    subset="training",
    seed=2,
)

In [ ]:
val_datagen = val_datagen.flow_from_dataframe(
    actor_4_dataset,
    directory=extracted_frames_path,
    x_col="frame_name",
    y_col="macro_labels",
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    subset="validation",
    seed=3,
)

### Build the model to fine tune

In [ ]:
IMG_SIZE = (224, 224, 3)

In [ ]:
feature_extractor = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=IMG_SIZE,
    pooling="avg",
)

feature_extractor.summary()

In [ ]:
# freeze all layers except last five
for layer in feature_extractor.layers[:-5]:
    layer.trainable = False

for i, layer in enumerate(feature_extractor.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
x = feature_extractor.output
x = Flatten()(x)
x = Dense(units=512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(units=256, activation="relu")(x)
x = Dense(units=3, activation="softmax")(x)

transfer_model = Model(inputs=feature_extractor.input, outputs=x)

transfer_model.summary()

In [ ]:
transfer_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics="accuracy"
)

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f"{projectdir}/vision/model_checkpoints/",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.EarlyStopping(patience=20),
    model_checkpoint,
]

In [ ]:
history = transfer_model.fit(train_datagen, validation_data = val_datagen, epochs=100, callbacks=callbacks)

In [ ]:
##plot metrics